In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [3]:
cs = pd.read_csv('C:\\Users\\HP\\Downloads\\Customer.csv')
pr = pd.read_csv('C:\\Users\\HP\\Downloads\\Products.csv')
od = pd.read_csv('C:\\Users\\HP\\Downloads\\Orders.csv')
tr = pd.read_csv('C:\\Users\\HP\\Downloads\\Transactions.csv')
rt = pd.read_csv('C:\\Users\\HP\\Downloads\\Ratings.csv')
sp = pd.read_csv('C:\\Users\\HP\\Downloads\\Subscription_Plan.csv')
sd = pd.read_csv('C:\\Users\\HP\\Downloads\\Subscription_Details.csv')
dv = pd.read_csv('C:\\Users\\HP\\Downloads\\Delievry.csv')
rr = pd.read_csv('C:\\Users\\HP\\Downloads\\Returns_Refund.csv')


In [5]:

df = od.merge(cs, on='C_ID', how='left') \
       .merge(pr, on='P_ID', how='left') \
       .merge(tr, on='Or_ID', how='left') \
       .merge(dv, on='DP_ID', how='left') \
       .merge(rr[['Or_ID', 'Return/Refund']], on='Or_ID', how='left')


df['Return/Refund'] = df['Return/Refund'].fillna('No')


df['Refund_Flag'] = df['Return/Refund'].map({'Refund': 1, 'Return': 0})


In [21]:

df = df.dropna(subset=['Refund_Flag'])


In [23]:
print(df)

             Or_ID         C_ID         P_ID  Order_Date Order_Time  Qty  \
0      OR_31000001  CS_11005317  PD_21001301  2024-02-27   22:02:00    1   
4      OR_31000004  CS_11004079  PD_21007443  2023-05-26   03:15:48   10   
11     OR_31000009  CS_11005938  PD_21001544  2024-06-03   13:21:46    5   
15     OR_31000012  CS_11004980  PD_21009453  2024-03-20   06:00:29    7   
16     OR_31000013  CS_11007513  PD_21003896  2023-06-13   18:51:21    5   
...            ...          ...          ...         ...        ...  ...   
15424  OR_31009973  CS_11004241  PD_21000276  2024-05-12   07:28:13    9   
15425  OR_31009974  CS_11005030  PD_21000572  2023-09-11   17:15:10    8   
15428  OR_31009976  CS_11008913  PD_21006012  2024-05-13   23:14:01    5   
15432  OR_31009979  CS_11000528  PD_21009483  2023-12-04   06:19:38    9   
15433  OR_31009979  CS_11000528  PD_21009483  2023-12-04   06:19:38    9   

              Coupon        DP_ID           C_Name  Gender  ...  \
0                NaN

In [25]:

features = df.drop(['Or_ID', 'C_ID', 'Return/Refund', 'Refund_Flag'], axis=1)


for col in features.select_dtypes(include='object').columns:
    features[col] = LabelEncoder().fit_transform(features[col].astype(str))

X = features
y = df['Refund_Flag']


In [27]:
df_processed = X.copy()
df_processed['Refund_Flag'] = y
df_processed.head()


,P_ID,Order_Date,Order_Time,Qty,Coupon,DP_ID,C_Name,Gender,Age,City,...,PName,Category,Specs1,Price,Tr_ID,Transaction_Mode,Tran_Status,DP_Name,DP_Ratings,Refund_Flag
0,220,392,1659,1,595,86,219,1,67,12,...,413,3,26,330.13,828,3,1,99,3.2,1.0
4,1227,135,239,10,16,291,726,0,57,5,...,587,4,215,234.48,965,1,0,41,3.5,1.0
11,257,481,998,5,595,17,1655,0,58,1,...,378,2,484,214.93,121,2,0,72,4.1,0.0
15,1550,412,457,7,595,187,1223,1,44,9,...,390,1,18,319.74,89,4,0,52,3.5,0.0
16,677,151,1417,5,250,40,1476,1,24,2,...,242,2,579,188.40,673,3,0,181,3.1,0.0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [35]:
y_pred = model.predict(X_test)


In [37]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7117263843648208
[[196  94]
 [ 83 241]]
              precision    recall  f1-score   support

         0.0       0.70      0.68      0.69       290
         1.0       0.72      0.74      0.73       324

    accuracy                           0.71       614
   macro avg       0.71      0.71      0.71       614
weighted avg       0.71      0.71      0.71       614

